In [1]:
import os

In [2]:
%pwd

'e:\\chicken_disease_classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\chicken_disease_classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir:Path
    tensorboard_root_log_dir:Path
    checkpoint_model_filepath:Path

In [7]:
from src.cnnClassifier.utils.common import read_yaml, created_directories
from cnnClassifier.constants import *


class ConfigurationCallbacksManager:
    def __init__(

        self,

        config_filepth=CONFIG_FILE_PATH,

        params_filepath=PARAMS_FILE_PATH,

    ):

        self.config = read_yaml(config_filepth)

        self.params = read_yaml(params_filepath)

        created_directories([self.config.artifacts_root])


    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:

        config = self.config.prepare_callbacks

        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)


        # creating dir
        created_directories(

            [Path(model_ckpt_dir), Path(config.tensorboard_root_log_dir)]
        )


        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath),
        )

        return prepare_callbacks_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

[2023-12-31 22:35:55,668:WARNING:module_wrapper:From e:\chicken_disease_classification\chicken_ven\lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]


In [9]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(self.config.tensorboard_root_log_dir)
        f"tb_logs_at_{timestamp}"

        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath), save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [self.create_ckpt_callbacks, self.create_tb_callbacks]

In [10]:
# pipeline
try:
    config = ConfigurationCallbacksManager()
    prepare_calback_config = config.get_prepare_callback_config()
    prepare_callback = PrepareCallback(config=prepare_calback_config)
    callback_list = prepare_callback.get_tb_ckpt_callbacks()

except Exception as e:
    raise e

[2023-12-31 22:35:56,597:INFO:common:yaml file:config\config.yaml loadded successfully]
[2023-12-31 22:35:56,601:INFO:common:yaml file:params.yaml loadded successfully]
[2023-12-31 22:35:56,604:INFO:common:Created directory as:{path}]
[2023-12-31 22:35:56,607:INFO:common:Created directory as:{path}]
[2023-12-31 22:35:56,609:INFO:common:Created directory as:{path}]


In [13]:
import time
timestamp=time.strftime("%Y-%m-%d-%H-%M-%S")
f"tb_logs_at {timestamp}"

'tb_logs_at 2023-12-31-22-38-15'